# Mini GPT Project

# Gerekli paketleri yükle

In [15]:
!pip install torch tqdm

In [16]:
import os, math, time
import torch
import torch.nn as nn
from torch.nn import functional as F
from tqdm import tqdm

# GPU ayarı

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Kullanılan cihaz:", device)

Kullanılan cihaz: cuda


# HYPERPARAMETERS

In [18]:
batch_size = 32
block_size = 128
max_iters = 5000
eval_interval = 200
learning_rate = 3e-4
eval_iters = 100
n_embed = 256
n_heads = 8
n_layers = 8
dropout = 0.1

# Veri (input.txt)

In [19]:
data_path = "input.txt"
if not os.path.exists(data_path):
    # Küçük bir Shakespeare corpus indir
    !wget -O input.txt https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

with open(data_path, "r", encoding="utf-8") as f:
    text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}

def encode(s):
    return [stoi[c] for c in s]

def decode(l):
    return "".join([itos[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data_source = train_data if split == 'train' else val_data
    ix = torch.randint(len(data_source) - block_size, (batch_size,))
    x = torch.stack([data_source[i:i+block_size] for i in ix])
    y = torch.stack([data_source[i+1:i+block_size+1] for i in ix])
    return x.to(device), y.to(device)

# Transformer Blocks

In [20]:
class SelfAttentionHead(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embed, head_size, bias=False)
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.value = nn.Linear(n_embed, head_size, bias=False)
        # Mask is now in MultiHeadAttention, removed from here
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # Input x is expected to be (B, T, head_size) from MultiHeadAttention after splitting heads
        B,T,C = x.size()
        k = self.key(x) # (B, T, head_size)
        q = self.query(x) # (B, T, head_size)
        v = self.value(x) # (B, T, head_size)

        # Scaled Dot-Product Attention
        att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1))) # (B, T, T)

        # Masking is now done in MultiHeadAttention

        att = F.softmax(att, dim=-1) # (B, T, T)
        att = self.dropout(att)

        out = att @ v # (B, T, T) @ (B, T, head_size) -> (B, T, head_size)

        return out


class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        # Define and register the mask here
        self.register_buffer("mask", torch.tril(torch.ones(block_size, block_size)).unsqueeze(0).unsqueeze(0))

        self.heads = nn.ModuleList([SelfAttentionHead(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size*num_heads, n_embed)
        self.dropout = nn.Dropout(dropout)
        self.head_size = head_size # Store head_size
        self.num_heads = num_heads # Store num_heads

    def forward(self, x):
        # Input x is expected to be (B, T, n_embed)
        B, T, C = x.size()

        # Apply Q, K, V linear layers and then split heads
        qkv = nn.Linear(n_embed, 3 * n_embed, bias=False).to(x.device)(x) # (B, T, 3 * n_embed)
        q, k, v = qkv.chunk(3, dim=-1) # Each (B, T, n_embed)

        # Split heads
        k = k.view(B, T, self.num_heads, self.head_size).transpose(1, 2) # (B, num_heads, T, head_size)
        q = q.view(B, T, self.num_heads, self.head_size).transpose(1, 2) # (B, num_heads, T, head_size)
        v = v.view(B, T, self.num_heads, self.head_size).transpose(1, 2) # (B, num_heads, T, head_size)

        # Scaled Dot-Product Attention
        # (q @ k.transpose(-2, -1)) * (1.0 / sqrt(head_size))
        att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(self.head_size)) # (B, num_heads, T, T)

        # Apply mask
        mask_sliced = self.mask[:, :, :T, :T]
        att = att.masked_fill(mask_sliced == 0, float('-inf'))

        att = F.softmax(att, dim=-1) # (B, num_heads, T, T)
        att = self.dropout(att)

        # Apply attention to values
        out = att @ v # (B, num_heads, T, head_size)

        # Concatenate heads
        out = out.transpose(1, 2).contiguous().view(B, T, self.num_heads * self.head_size) # (B, T, n_embed)

        # Linear projection
        out = self.dropout(self.proj(out)) # (B, T, n_embed)

        return out


class FeedForward(nn.Module):
    def __init__(self, n_embed):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embed, 4 * n_embed),
            nn.GELU(),
            nn.Linear(4 * n_embed, n_embed),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        # Input and output shape should be (B, T, n_embed)
        return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embed, n_heads):
        super().__init__()
        head_size = n_embed // n_heads
        self.sa = MultiHeadAttention(n_heads, head_size)
        self.ff = FeedForward(n_embed)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)

    def forward(self, x):
        # Input and output shape should be (B, T, n_embed)
        # Residual connection 1
        x = x + self.sa(self.ln1(x)) # (B, T, n_embed) + (B, T, n_embed) -> (B, T, n_embed)

        # Residual connection 2
        x = x + self.ff(self.ln2(x)) # (B, T, n_embed) + (B, T, n_embed) -> (B, T, n_embed)
        return x

# Model

In [21]:
class MiniGPT(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.position_embedding_table = nn.Embedding(block_size, n_embed)
        self.blocks = nn.Sequential(*[Block(n_embed, n_heads) for _ in range(n_layers)])
        self.ln_f = nn.LayerNorm(n_embed)
        self.head = nn.Linear(n_embed, vocab_size)

    def forward(self, idx, targets=None):
        B,T = idx.size()
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=idx.device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.head(x)

        if targets is None:
            loss = None
        else:
            B,T,C = logits.size()
            loss = F.cross_entropy(logits.view(B*T,C), targets.view(B*T))

        return logits, loss

    def generate(self, idx, max_new_tokens, temperature=1.0, top_k=None):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :] / temperature

            if top_k is not None:
                v, _ = torch.topk(logits, top_k)
                minv = v[:, -1].unsqueeze(1)
                logits = torch.where(logits < minv, torch.full_like(logits, -1e10), logits)

            probs = F.softmax(logits, dim=-1)
            next_id = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, next_id), dim=1)
        return idx

model = MiniGPT().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Loss tahmini

In [22]:
@torch.no_grad()
def estimate_loss():
    model.eval()
    out = {}
    for split in ['train','val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            xb,yb = get_batch(split)
            _, loss = model(xb, yb)
            losses[k] = loss.item()
        out[split] = losses.mean().item()
    model.train()
    return out

# Training

In [23]:
start_time = time.time()
best_val = 1e9
for iter in range(max_iters):
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter} | train loss {losses['train']:.4f} | val loss {losses['val']:.4f} | time {(time.time()-start_time):.1f}s")
        context = torch.zeros((1,1), dtype=torch.long, device=device)
        sample = model.generate(context, max_new_tokens=200, temperature=1.0, top_k=10)[0].tolist()
        print("---- sample ----")
        print(decode(sample))
        print("----------------")
    if losses['val'] < best_val:
        best_val = losses['val']
        torch.save(model.state_dict(), "mini_gpt_checkpoint.pt")
    xb,yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()

print("Training finished.")
torch.save(model.state_dict(), "mini_gpt_final.pt")

step 0 | train loss 4.3726 | val loss 4.3639 | time 6.1s
---- sample ----

eCeNW:KlSQbRuSg$$TzZDSRfzvw;WGoNOLGoXSLSszIvXYPQCKduvHO&WxrmZ3YzDIFJpbNwnA!FWBnAMPtghbgivZqwpFEHSWTkgegiyiV?ivZFRu-,qoIDcMa,ixcf3
G;W,L,Ee!KPItm3e:ILdaIEL!m3Gw;Im?bGF,Lm!mEP?PI?!;,PGGGdG$eJEKWGF?hmKWG
----------------
step 200 | train loss 2.4983 | val loss 2.5149 | time 31.4s
---- sample ----



Wat l be s thourou thealathire thanof berd sthinoupomesth therthanghit mers me th tare nt anofofo s mend,
AMINCEETor d.
INGon wh thind hatth o ded t weathat, she tins mousoureres asitheng, owere of 
----------------
step 400 | train loss 2.4795 | val loss 2.4985 | time 57.0s
---- sample ----

Th ayouthy h athon, we core he t thowithe I fowadiss se an breren he the, hiroun, ot wa hin tomande wis ime ped tayo cele he hato th, hes se whisthees, morte son wh tinter he s co ssthat o ise ht te m
----------------
step 600 | train loss 2.4711 | val loss 2.4950 | time 82.1s
---- sample ----


INI imeave t fomothusathes thishas h

# Text üretme fonksiyonu

In [24]:
def generate_from_file(prefix, length=400, temp=1.0, top_k=20):
  model.eval()
  context = torch.tensor(encode(prefix), device=device, dtype=torch.long).unsqueeze(0)
  out_ids = model.generate(context, max_new_tokens=length, temperature=temp, top_k=top_k)[0].tolist()
  return decode(out_ids)

print("\n--- Final sample generation ---")
sample_text = generate_from_file("The", length=300, temp=0.8, top_k=10)
print(sample_text)


--- Final sample generation ---
The has thens,
HAnd, woote all sh alothe heng h sensered at bet mitheree s, for f se is, thary s thoureenowangusen, thathis the t waras that totore taith artowond tour, aveane tr boreld fo fris, t the ts ang manoms s me owir me tind stofte hellathadis te thend, o myof hieno he allisthowin asis byor s t


# Çıktıyı İyileştirmek İçin Yapılabilecekler


1. **Model Boyutunu Artırma**

   * `n_embed`: **128 → 256 veya 384**
   * `n_heads`: **4 → 6 veya 8**
   * `n_layers`: **4 → 6 veya 8**

2. **Daha Fazla Eğitim**

   * `max_iters`: **2000 → 5000–10000**

3. **Öğrenme Oranını Ayarlama**

   * `learning_rate`: **3e-4 (mevcut)**

     * Alternatif denemeler: **1e-4, 2e-4, 5e-4**

4. **Daha Büyük Veri Kümesi Kullanma**

   * Tiny Shakespeare (~1MB) yerine:

     * **Wikipedia makaleleri (10–100MB)**
     * **Kitaplar (Project Gutenberg, birkaç MB–GB)**
     * **Kendi notlarınız / domain spesifik veriler**

5. **Gelişmiş Eğitim Teknikleri**

   * **LR Scheduler**: `torch.optim.lr_scheduler.CosineAnnealingLR`

     * Örn: başlangıç **3e-4**, minimum **1e-5**
   * **Weight Decay (düzenlileştirme)**:

     * `optimizer = AdamW(model.parameters(), lr=3e-4, weight_decay=1e-2)`


# Kaynaklar

https://github.com/karpathy/nanoGPT

https://huggingface.co/docs/transformers/en/index

https://chatgpt.com/